### PV Secondary Target Selection

## sga_semimajor_grid
### Method to create the secondary targeting file for the intermediate points on the semi-major axis of large galaxies (from the SGA) in the BGS sample

##### Author: Kelly Douglass (University of Rochester)

See `/project/projectdirs/desi/target/secondary/README` for output data model

### Target classes
1. End points on the major axis
2. **Points along the major axis**
3. Points off-axis

In [18]:
from astropy.table import Table, Column
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord

import numpy as np

## Parameters

In [24]:
OVERRIDE = True
REF_EPOCH = 2015.5

#output_directory = '/project/projectdirs/desi/target/secondary/indata/'
output_directory = '../'

## Target catalogs

[Siena Galaxy Atlas](https://portal.nersc.gov/project/cosmo/temp/ioannis/SGA-html-dr9alpha/SGA-2arcmin.html)

In [7]:
# Target catalog file names

input_directory = '/Users/kellydouglass/Documents/Research/data/SGA/'

input_filename = input_directory + 'SGA-ellipse-v3.0.kd.fits'

hdu = fits.open(input_filename)
large_galaxies = hdu[1].data
hdu.close()

In [10]:
large_galaxies.names

['SGA_ID',
 'GALAXY',
 'PGC',
 'SGA_RA',
 'SGA_DEC',
 'MORPHTYPE',
 'PA_LEDA',
 'D25_LEDA',
 'BA_LEDA',
 'Z_LEDA',
 'SB_D25_LEDA',
 'MAG_LEDA',
 'BYHAND',
 'REF',
 'IN_FOOTPRINT',
 'IN_FOOTPRINT_GRZ',
 'GROUP_ID',
 'GROUP_NAME',
 'GROUP_MULT',
 'GROUP_PRIMARY',
 'GROUP_RA',
 'GROUP_DEC',
 'GROUP_DIAMETER',
 'BRICKNAME',
 'DIAM',
 'DIAM_REF',
 'PA',
 'BA',
 'ELLIPSEBIT',
 'DROPBIT',
 'RELEASE',
 'BRICKID',
 'OBJID',
 'BRICK_PRIMARY',
 'MASKBITS',
 'ITERATIVE',
 'TYPE',
 'RA',
 'DEC',
 'RA_IVAR',
 'DEC_IVAR',
 'BX',
 'BY',
 'DCHISQ',
 'EBV',
 'MJD_MIN',
 'MJD_MAX',
 'REF_CAT',
 'REF_ID',
 'PMRA',
 'PMDEC',
 'PARALLAX',
 'PMRA_IVAR',
 'PMDEC_IVAR',
 'PARALLAX_IVAR',
 'REF_EPOCH',
 'GAIA_POINTSOURCE',
 'GAIA_PHOT_G_MEAN_MAG',
 'GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR',
 'GAIA_PHOT_G_N_OBS',
 'GAIA_PHOT_BP_MEAN_MAG',
 'GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR',
 'GAIA_PHOT_BP_N_OBS',
 'GAIA_PHOT_RP_MEAN_MAG',
 'GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR',
 'GAIA_PHOT_RP_N_OBS',
 'GAIA_PHOT_VARIABLE_FLAG',
 'GAIA_

## Set second priority: points along the major axis

Note: Galaxy centers should already be BGS targets

### Within the SGA

We require all galaxies to be within the DESI footprint.

In [11]:
SGA_galaxies = large_galaxies[large_galaxies['IN_FOOTPRINT_GRZ']]

### Large galaxies

We target only those galaxies with $D(26) > 1.4'$ (the fiber patrol radius).

In [14]:
max_patrol_radius = 1.4 # arcminutes

lg_SGA = SGA_galaxies[SGA_galaxies['DIAM'] >= 2*max_patrol_radius]

print(len(lg_SGA))

3074


## Calculate (ra, dec) of major axis end points

We set fiber target locations along the major axis at a distance of $xR_e$, where $x$ varies from 0.2 to 1 in steps of 0.2.

In [19]:
center_ra = lg_SGA['RA']   # degrees
center_dec = lg_SGA['DEC'] # degrees

centers = SkyCoord(center_ra*u.deg, center_dec*u.deg)

phi = lg_SGA['PA']*u.deg
r26 = 0.5*lg_SGA['DIAM']*u.arcmin

In [46]:
x = np.arange(0.2,1.2,0.2).reshape((1,5))

# Distances along the semi-major axis from the center coordinate for our targets
delta_a = np.dot(r26.reshape((len(r26),1)),x)
delta_a = delta_a.T

# Target positions
fiber1 = centers.directional_offset_by(phi, delta_a)
fiber2 = centers.directional_offset_by(phi + 180*u.deg, delta_a)

fiber_ra = np.concatenate((np.ravel(fiber1.ra), np.ravel(fiber2.ra)))
fiber_dec = np.concatenate((np.ravel(fiber1.dec), np.ravel(fiber2.dec)))

In [47]:
print(r26[0])
print(delta_a[:,0])

2.4168472290039062 arcmin
[0.48336945 0.96673889 1.45010834 1.93347778 2.41684723] arcmin


### Write target list to file

In [48]:
N_targets = len(fiber_ra)

In [49]:
lg_gal_grid = Table([Column(fiber_ra, name='RA'), 
                     Column(fiber_dec, name='DEC'), 
                     Column(np.zeros(N_targets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(N_targets, dtype='>f4'), name='PMDEC'), 
                     Column(REF_EPOCH*np.ones(N_targets, dtype='>f4'), name='REF_EPOCH'),
                     Column(OVERRIDE*np.ones(N_targets, dtype='bool'), name='OVERRIDE')])

In [51]:
lg_gal_grid.write(output_directory + 'sga_semimajor_grid.fits', format='fits', overwrite=True)

## Target statistics

In [52]:
print('The number of large galaxies is', len(lg_SGA))
print('The number of targets is', N_targets)

sky_chunk_boolean = np.logical_and.reduce([lg_gal_grid['RA'] > 150, lg_gal_grid['RA'] < 250, 
                                           lg_gal_grid['DEC'] > 0, lg_gal_grid['DEC'] < 50])

num_targets_in_sky_chunk = np.sum(sky_chunk_boolean)

sky_area = 100*50

print('The number of fiber placements in this portion of the sky is', 
      num_targets_in_sky_chunk)
print('The number of fiber placements per square degree is', 
      num_targets_in_sky_chunk/sky_area)

The number of large galaxies is 3074
The number of targets is 30740
The number of fiber placements in this portion of the sky is 9488
The number of fiber placements per square degree is 1.8976
